In [ ]:
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import numpy as np

In [ ]:
import numpy as np
import pandas as pd


class Metrics:
    @staticmethod
    def k_fold_cross_validation(input_dataset, expected_output, k: int, df_columns: list = None):
        if k <= 0 or k > len(input_dataset):
            raise ValueError(
                "k must be greater than 0 and less than the number of rows in the dataset")

        if len(input_dataset) != len(expected_output):
            raise ValueError(
                "The number of rows in the dataset must be equal to the number of rows in the expected output")

        # Shuffle the dataset
        shuffled_tuple = list(zip(input_dataset, expected_output))
        np.random.shuffle(shuffled_tuple)
        shuffled_input_dataset, shuffled_expected_output = map(
            np.array, zip(*shuffled_tuple))

        items_per_fold = int(len(input_dataset) / k)
        folds = []

        print(shuffled_input_dataset[0:2])
        print("============")
        print(shuffled_expected_output[0:2])

        # Split the dataset into k folds
        for i in range(k):
            test_set = shuffled_input_dataset[i *
                                     items_per_fold: (i + 1) * items_per_fold]
            test_set_expected_output = shuffled_expected_output[i *
                                                                items_per_fold: (i + 1) * items_per_fold]

            train_set = np.concatenate(
                [shuffled_input_dataset[:i * items_per_fold],
                 shuffled_input_dataset[(i + 1) * items_per_fold:]])
                
            train_set_expected_output = np.concatenate(
                [shuffled_expected_output[:i * items_per_fold],
                 shuffled_expected_output[(i + 1) * items_per_fold:]])
                
            # If df_columns is not None, then we need to create a dataframe for both sets
            if df_columns is not None:
                train_set = pd.DataFrame(
                    train_set, columns=df_columns)
                test_set = pd.DataFrame(
                    test_set, columns=df_columns)
                train_set_expected_output = pd.DataFrame(
                    train_set_expected_output, columns=['expected'])
                test_set_expected_output = pd.DataFrame(
                    test_set_expected_output, columns=['expected'])
                
            # Load the test and train sets into the folds
            folds.append({
                'train_set': train_set,
                'test_set': test_set,
                'train_set_expected_output': train_set_expected_output,
                'test_set_expected_output': test_set_expected_output,
            })


        return folds

    @staticmethod
    def k_fold_cross_validation_eval(input_dataset, expected_output, model, k: int, df_columns: list = None):
        if model is None:
            raise ValueError("Model cannot be None")

        folds = Metrics.k_fold_cross_validation(
            input_dataset, expected_output, k=k, df_columns=df_columns)

        # Evaluate the model on each fold
        results = []
        for fold in folds:
            # Train the model, on this case the expected output is present on the train set
            # so ignore the train_set_expected_output field
            train_set = fold['train_set']
            model.train(train_set)

            # Evaluate the model on the test set
            test_set = fold['test_set']
            results.append(model.test(test_set))

        return results


In [ ]:
class NaiveBayesTextClassifier():
    predicted_class_column_name = 'prediccion'

    def __init__(self, classes, text_column_name, classes_column_name, tokenizer):
        self.relative_frequencies = {}  # by_class_by_value
        self.classes_probabilities = {}
        self.classes = classes
        self.classes_column_name = classes_column_name
        self.text_column_name = text_column_name
        self.row_count_by_class = {}
        self.tokenizer = tokenizer

    def train(self, data_df):
        for c in self.classes:
            class_df = data_df[data_df[self.classes_column_name] == c]
            self.row_count_by_class[c] = len(class_df)
            self.classes_probabilities[c] = len(class_df) / len(data_df)

            values_appearances = {}
            # iterate over rows within class
            for i in range(len(class_df)):
                row = class_df.iloc[[i]]

                tokenized_text = self.tokenizer(
                    row[self.text_column_name].values[0])
                # iterate over values within row
                for token in tokenized_text:
                    # initialize possible value if not present in map, otherwise increment appereances
                    if token not in values_appearances:
                        values_appearances[token] = 1
                    else:
                        values_appearances[token] += 1

            # calculate relative frequencies
            self.relative_frequencies[c] = {token: (token_count + 1) / (len(class_df) + len(
                self.classes)) for token, token_count in values_appearances.items()}

    def classify(self, sample):
        tokenized_sample = self.tokenizer(sample)
        classification = {}

        maximizing_class = None
        maximizing_prod = -1
        for c in self.classes:

            prod = self.classes_probabilities[c]
            token_likelihoods = self.relative_frequencies[c]
            laplace_constant = 1 / \
                (self.row_count_by_class[c] + len(self.classes))

            for token in tokenized_sample:
                prod *= token_likelihoods[token] if token in token_likelihoods else laplace_constant

            classification[c] = prod

        return dict(sorted(classification.items(), key=lambda item: item[1], reverse=True))

    def test(self, test_df):
        predicted_classes = []

        for i in range(len(test_df)):
            row = test_df.iloc[[i]]
            row_class = row[self.classes_column_name].values[0] 

            classification = self.classify(row[self.text_column_name].values[0])
            predicted_class = max(classification, key=classification.get)

            predicted_classes.append(predicted_class)

        # append results column to new dataframe
        results_df = test_df.copy()
        results_df[self.predicted_class_column_name] = predicted_classes
        
        return results_df
        
    def get_confusion_matrix(self):
        pass

    def get_evaluation_metrics_by_class(self, test_results_df):
        pass

    def plot_roc_curve(self):
        pass


In [ ]:
def tokenize(text: str):
    return text.split()


In [ ]:
data_df = pd.read_csv("./Noticias_argentinas.txt", header=0, sep='\t')
classes = ["Economia", "Salud", "Ciencia y Tecnologia", "Deportes"]

data_df = data_df[data_df["categoria"].isin(classes)]


nbclassifier = NaiveBayesTextClassifier(
    classes, "titular", "categoria", tokenize)
# Get expected output for dataset
expected_output = data_df["categoria"]
print(len(expected_output))
# print("Expected output: ", expected_output)
print(Metrics.k_fold_cross_validation_eval(data_df.values.tolist(), expected_output.values.tolist(), model=nbclassifier,df_columns=data_df.columns, k=2))

# train_set = data_df.sample(frac=0.8, random_state=1)
# print("======================")
# print(train_set.index)
# test_set = data_df.drop(train_set.index)

# nbclassifier.train(train_set)
# print(nbclassifier.test(test_set))

# sample_columns = data_df.columns.drop("categoria")

# print(nbclassifier.classify("Histórico: Los Pumas derrotaron por primera vez a los All Blacks en Nueva Zelanda"))
# print(nbclassifier.classify("Maradona negó haber criticado a Messi, disparó otra vez contra Scaloni y también la ligó Solari"))
# print(nbclassifier.classify("Guzmán"))




